# Using dfe_cont_store.generation.ai

This notebook demonstrates the usage of the `ai` module from the `dfe_cont_store.generation` package. 

## Overview

The `ai` module offers a set of functions to:
1. Create system, user, and assistant messages
2. Generate chat completions using OpenAI's models
3. Handle structured responses with custom JSON schemas/Pydantic objects

## Key Features

- **Automatic OpenAI Settings**: The module automatically creates a default OpenAI client using predefined settings, simplifying the setup process.
- **Flexible Message Creation**: Easily create system, user, and assistant messages for your conversations.
- **Structured Responses**: Support for generating responses in specific JSON formats using custom schemas.
- **Token Counting**: Automatic tracking of input and output token usage for each request.

## Prerequisites

- Ensure you have the `dfe_cont_store` package installed and properly configured.
- An active OpenAI API key should be set up in your environment variables or configuration files.

### Import necessary packages

In [1]:
from oak_hackaton.generation import ai 
import json
from typing import List

### Example simple usage

In [2]:
# Create messages 
sys_msg = ai.create_system_msg("You are a dog who can only response with 'woof'")
usr_msg = ai.create_user_msg("Hello chat GPT")

# Create a chat completion request 
response = ai.chat_completion(messages=[sys_msg, usr_msg])

# Print response 
print(f"Input tokens used = {response.input_token_count}")
print(f"Output tokens used = {response.output_token_count}")
print(f"Content response = {response.content}")

Input tokens used = 27
Output tokens used = 3
Content response = Woof!


In [3]:
# If we want to go back to the AI and keep the history, we can do this easily 
assistant_msg = ai.create_assistant_msg(response.content)
new_usr_msg = ai.create_user_msg("Wow you really are a dog")
new_response = ai.chat_completion(messages = [sys_msg, usr_msg, assistant_msg, new_usr_msg])

# Print response 
print(f"Input tokens used = {new_response.input_token_count}")
print(f"Output tokens used = {new_response.output_token_count}")
print(f"Content response = {new_response.content}")

Input tokens used = 44
Output tokens used = 5
Content response = Woof woof!


### Example using json schema

In [4]:
# Create messages 
sys_msg = ai.create_system_msg("You are a helpful assistant")
usr_msg = ai.create_user_msg("What are the constituent countries of the UK and their capitals?")

# Create a json schema 
json_schema = {
    "name": "uk_countries",
    "schema": {
        "type": "object",
        "properties": {
            "countries": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "capital": {"type": "string"}
                    },
                    "required": ["name", "capital"],
                    "additionalProperties": False
                }
            }
        },
        "required": ["countries"],
        "additionalProperties": False
    },
    "strict": True
}

# Create a chat completion request 
response = ai.chat_completion(messages=[sys_msg, usr_msg], structured_response = True, json_schema=json_schema)

# Parse the content as JSON
parsed_content = json.loads(response.content)

# Print the JSON in a formatted way
print("Content response:")
print(json.dumps(parsed_content, indent=2))

Content response:
{
  "countries": [
    {
      "name": "England",
      "capital": "London"
    },
    {
      "name": "Scotland",
      "capital": "Edinburgh"
    },
    {
      "name": "Wales",
      "capital": "Cardiff"
    },
    {
      "name": "Northern Ireland",
      "capital": "Belfast"
    }
  ]
}


### Same example using a Pydantic model

In [5]:
# Define a Pydantic model for a country
class Country(BaseModel):
    name: str
    capital: str

# Define a Pydantic model for a list of countries
class UKCountries(BaseModel):
    countries: List[Country]

# Create a chat completion request 
response = ai.chat_completion(messages=[sys_msg, usr_msg], structured_response = True, pydantic_model=UKCountries)

# Convert the pydantic model to a json string
print("Content response:")
print(response.content.model_dump_json(indent=2))


Content response:
{
  "countries": [
    {
      "name": "England",
      "capital": "London"
    },
    {
      "name": "Scotland",
      "capital": "Edinburgh"
    },
    {
      "name": "Wales",
      "capital": "Cardiff"
    },
    {
      "name": "Northern Ireland",
      "capital": "Belfast"
    }
  ]
}
